In [1]:
import pandas as pd
import torch 

df=pd.read_csv("train_clean_kalman.csv")
df['signal']=df['signal']/df['signal'].max()

In [2]:
df['open_channels'].max()

10

# Create time series dataset

In [3]:
class TimesereisDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.Series(), seqLen=10):
        super().__init__()
        self.df=df
        self.seqLen=seqLen
    
    def __len__(self):
        return self.df.shape[0]-self.seqLen-1
    
    def __getitem__(self,index):
        x=self.df.iloc[index:index+self.seqLen,1].values
        y=self.df.iloc[index+self.seqLen-1,2] 
        return x,y   
seqLen=50
timeSeriesDataset = TimesereisDataset(df,seqLen)

ipykernel_launcher:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [4]:
trainNumbers=int(len(timeSeriesDataset)*0.9)
trainDataset,testDataset=torch.utils.data.random_split(timeSeriesDataset,[trainNumbers,len(timeSeriesDataset)-trainNumbers])
trainDataLoader=torch.utils.data.DataLoader(trainDataset,batch_size=8,shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=8,shuffle=True)

In [5]:

from Wavenet import WaveNet,WaveNetClassifier
from tqdm import tqdm 

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="cpu"
wavenetClassifierModel=WaveNetClassifier(seqLen,df['open_channels'].max()+1)
wavenetClassifierModel.to(device)

wavenetClassifierModel.train()

optimizer=torch.optim.AdamW(wavenetClassifierModel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
lossFunction = torch.nn.CrossEntropyLoss()

def calc_accuracy(Out,Y):
    max_vals, max_indices = torch.max(Out,1)
    train_acc = (max_indices == Y).sum().item()/max_indices.size()[0]
    return train_acc
  
epochs=1
globalStep=500

for epoch in range(epochs):
    for step, (x_train,y_train) in tqdm(enumerate(trainDataLoader),desc="Training"):
         x_train = torch.unsqueeze(x_train,dim=1).float()
         x_train.to(device)
         y_train.to(device)
         output=wavenetClassifierModel(x_train)
         output = torch.squeeze(output,dim=1)

         loss= lossFunction(output,y_train)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         if step%globalStep==0:
            # scheduler.step()
            # print(output.detach().numpy())
            # print(y_train.numpy())
            with torch.no_grad():
                accuracy=0
                loss=0
                for stepTest, (x_test,y_test) in tqdm(enumerate(testDataLoader),desc="Validation"):
                    x_test.to(device)
                    y_test.to(device)
                    x_test = torch.unsqueeze(x_test,dim=1).float()
                    output=wavenetClassifierModel(x_test)
                    output = torch.squeeze(output,dim=1)
                    accuracy+=calc_accuracy(output,y_test)*100
                    loss+= lossFunction(output,y_test).item()
                    if stepTest>200:
                        break
            print(f"loss for step {step} : {loss/stepTest}  :  Accuracy: {accuracy/stepTest} %")

         
    print(f"epch {epoch}")
         

Validation: 201it [00:01, 154.43it/s]
Training: 6it [00:01,  4.54it/s]

loss for step 0 : 2.3986626584731523  :  Accuracy: 17.412935323383085 %


Validation: 201it [00:01, 179.46it/s]
Training: 505it [00:11, 13.23it/s]

loss for step 500 : 2.085301135902974  :  Accuracy: 52.67412935323383 %


Validation: 201it [00:01, 182.39it/s]
Training: 1005it [00:21, 13.41it/s]

...

Validation: 201it [00:01, 124.56it/s]
Training: 3507it [01:13, 11.11it/s]

loss for step 3500 : 1.9115325550534832  :  Accuracy: 65.17412935323384 %


Validation: 201it [00:01, 122.00it/s]
Training: 4005it [01:26, 10.08it/s]

loss for step 4000 : 1.9178816833306307  :  Accuracy: 64.55223880597015 %


Training: 4062it [01:28, 30.30it/s]